# 02_upload_to_hdfs.ipynb

Загрузка очищенных данных и изображений в HDFS для проекта анализа COVID-19 рентген-снимков.

В этом ноутбуке мы:
- загружаем файл `metadata_cleaned.csv`, подготовленный на предыдущем шаге;
- настраиваем пути к локальным данным и к HDFS;
- создаём в HDFS структуру каталогов `/covid_dataset/...`;
- загружаем в HDFS очищенные метаданные (CSV и Parquet);
- загружаем изображения в HDFS, раскладывая их по подкаталогам в зависимости от диагноза (COVID19 / PNEUMONIA / NORMAL / OTHER).

**Предположения:**
- ноутбук запускается в окружении, где доступна команда `hdfs` (например, внутри Docker-кластера с Hadoop + Spark);
- рядом с этим ноутбуком лежат файлы `metadata_cleaned.csv` и `metadata.parquet`;
- папка с изображениями `images/` находится по указанному ниже пути (его можно изменить под свою структуру).


In [7]:
import os                   # Модуль для работы с файловой системой (пути к файлам и папкам)
import subprocess           # Модуль для запуска системных команд (будем вызывать hdfs dfs)
import pandas as pd         # Библиотека для работы с табличными данными (DataFrame)
import shutil  # модуль для работы с путями к бинарникам (ищет, где лежит исполняемый файл)

# Проверяем, доступна ли в системе команда 'hdfs'
HDFS_BIN = shutil.which("hdfs")  # вернёт путь к бинарнику 'hdfs' или None, если его нет

if HDFS_BIN is None:
    print("⚠️ Команда 'hdfs' не найдена в системе. Все HDFS-команды будут только печататься (без реального выполнения).")
else:
    print("Найден бинарник hdfs по пути:", HDFS_BIN)


⚠️ Команда 'hdfs' не найдена в системе. Все HDFS-команды будут только печататься (без реального выполнения).


In [8]:
# Локальные пути (относительно папки, где лежит этот ноутбук)
LOCAL_METADATA_CSV_PATH = "metadata_cleaned.csv"   # путь к очищенному CSV с метаданными
LOCAL_METADATA_PARQUET_PATH = "metadata.parquet"   # путь к очищенному Parquet-файлу
LOCAL_IMAGES_DIR = "images"                        # папка с изображениями рентген-снимков

# Корневой путь в HDFS, где будет храниться наш датасет
HDFS_BASE_DIR = "/covid_dataset"                   # основной каталог для проекта в HDFS

# Подкаталоги внутри HDFS
HDFS_METADATA_DIR = f"{HDFS_BASE_DIR}/metadata"    # сюда положим метаданные (CSV и Parquet)
HDFS_IMAGES_DIR = f"{HDFS_BASE_DIR}/images"        # сюда положим изображения по категориям
HDFS_PROCESSED_DIR = f"{HDFS_BASE_DIR}/processed"  # сюда будем складывать будущие производные данные (если понадобятся)


In [9]:
# Загружаем очищенный CSV-файл с метаданными
metadata_clean = pd.read_csv(LOCAL_METADATA_CSV_PATH)

# Показываем первые строки таблицы, чтобы убедиться, что всё читается корректно
metadata_clean.head()


,patientid,filename,age,age_filled,age_group,sex,view,finding,finding_unified,date
0,2,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,65.0,65.0,senior,M,PA,Pneumonia/Viral/COVID-19,COVID19,"January 22, 2020"
1,2,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,65.0,65.0,senior,M,PA,Pneumonia/Viral/COVID-19,COVID19,"January 25, 2020"
2,2,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,65.0,65.0,senior,M,PA,Pneumonia/Viral/COVID-19,COVID19,"January 27, 2020"
3,2,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,65.0,65.0,senior,M,PA,Pneumonia/Viral/COVID-19,COVID19,"January 28, 2020"
4,4,nejmc2001573_f1a.jpeg,52.0,52.0,adult,F,PA,Pneumonia/Viral/COVID-19,COVID19,"January 25, 2020"


In [10]:
def run_hdfs_command(command_list):
    """
    Вспомогательная функция для запуска команд hdfs dfs через subprocess.
    
    command_list: список строк, например ["hdfs", "dfs", "-ls", "/"].
    """
    print("Выполняем команду:", " ".join(command_list))  # печатаем команду, чтобы видеть, что происходит

    result = subprocess.run(          # запускаем внешнюю команду
        command_list,                 # список аргументов, например ["hdfs", "dfs", "-mkdir", "-p", "/covid_dataset"]
        stdout=subprocess.PIPE,       # перенаправляем стандартный вывод (stdout), чтобы потом прочитать его в Python
        stderr=subprocess.PIPE,       # перенаправляем стандартный вывод ошибок (stderr)
        text=True                     # говорим, что нужно работать со строками, а не с байтами (Python сам декодирует)
    )

    if result.stdout:                 # если команда что-то вывела в stdout
        print("STDOUT:")              # печатаем заголовок
        print(result.stdout)          # печатаем сам вывод

    if result.stderr:                 # если в stderr есть что-то (часто там варнинги или ошибки)
        print("STDERR:")              # печатаем заголовок
        print(result.stderr)          # печатаем текст ошибок/варнингов

    if result.returncode != 0:        # если код возврата команды не равен нулю, значит произошла ошибка
        print("Команда завершилась с ошибкой, код:", result.returncode)  # сообщаем об этом


In [17]:
import shutil  # модуль для копирования файлов и поиска бинарников

# Флаг: работаем ли мы с реальным HDFS (True) или эмулируем его локальными папками (False)
USE_HDFS = False
# Сейчас у тебя на ноуте нет команды hdfs, поэтому оставляем False.
# Если потом будешь запускать это в Docker-кластере с Hadoop — можно поставить True.

# Проверяем, доступна ли команда 'hdfs' в системе (на будущее)
HDFS_BIN = shutil.which("hdfs")  # вернёт строку с путём к бинарнику 'hdfs' или None, если его нет

if USE_HDFS:
    # Если мы хотим реальный HDFS
    if HDFS_BIN is None:
        print("⚠️ USE_HDFS=True, но команда 'hdfs' не найдена в PATH. Команды hdfs dfs будут падать.")
    else:
        print("Режим: реальный HDFS. Найден бинарник hdfs по пути:", HDFS_BIN)
else:
    # Если мы работаем в локальном режиме (эмуляция HDFS обычными папками)
    print("Режим: эмуляция HDFS через локальные папки.")


def hdfs_path_to_local(hdfs_path: str) -> str:
    """
    Преобразует HDFS-путь вида '/covid_dataset/metadata'
    в локальный путь 'covid_dataset/metadata', чтобы создавать такие же директории на диске.
    """
    # Если путь начинается с / (корень), отбрасываем первый символ
    if hdfs_path.startswith("/"):
        return hdfs_path[1:]
    # Если слэша нет, возвращаем строку как есть
    return hdfs_path


def make_dir(hdfs_dir_path: str):
    """
    Создаёт каталог:
    - либо в реальном HDFS (если USE_HDFS=True и есть команда hdfs),
    - либо как локальную папку (если USE_HDFS=False).
    """
    if USE_HDFS and HDFS_BIN is not None:
        # Реальный HDFS: строим команду hdfs dfs -mkdir -p <путь>
        cmd = ["hdfs", "dfs", "-mkdir", "-p", hdfs_dir_path]
        print("Создаём каталог в HDFS командой:", " ".join(cmd))
        # Запускаем внешнюю команду
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        # Если есть вывод в stdout — печатаем
        if result.stdout:
            print("STDOUT:")
            print(result.stdout)
        # Если есть вывод в stderr (ошибки/варнинги) — тоже печатаем
        if result.stderr:
            print("STDERR:")
            print(result.stderr)
    else:
        # Локальная эмуляция HDFS: создаём обычную папку в файловой системе
        local_dir = hdfs_path_to_local(hdfs_dir_path)  # переводим "/covid_dataset/..." → "covid_dataset/..."
        print("Создаём локальный каталог (эмуляция HDFS):", local_dir)
        os.makedirs(local_dir, exist_ok=True)          # создаём все промежуточные папки, если их ещё нет


def put_file(local_src_path: str, hdfs_dst_dir: str):
    """
    "Кладёт" файл:
    - либо в реальный HDFS (через hdfs dfs -put),
    - либо в локальную папку (копированием файла).
    """
    if USE_HDFS and HDFS_BIN is not None:
        # Реальный HDFS: команда hdfs dfs -put -f <local> <hdfs_dir>
        cmd = ["hdfs", "dfs", "-put", "-f", local_src_path, hdfs_dst_dir]
        print("Кладём файл в HDFS командой:", " ".join(cmd))
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.stdout:
            print("STDOUT:")
            print(result.stdout)
        if result.stderr:
            print("STDERR:")
            print(result.stderr)
    else:
        # Локальная эмуляция: просто копируем файл на диск в нужную папку
        local_dst_dir = hdfs_path_to_local(hdfs_dst_dir)      # "/covid_dataset/metadata" → "covid_dataset/metadata"
        os.makedirs(local_dst_dir, exist_ok=True)             # убеждаемся, что папка существует
        dst_path = os.path.join(local_dst_dir, os.path.basename(local_src_path))  # полный путь назначения
        print(f"Копируем файл {local_src_path} → {dst_path} (эмуляция HDFS)")
        shutil.copy2(local_src_path, dst_path)                # копируем файл, сохраняя метаданные (время и т.п.)


Режим: эмуляция HDFS через локальные папки.


In [18]:
# 1. Создаём корневой каталог /covid_dataset
make_dir(HDFS_BASE_DIR)          # будет создан локальный каталог "covid_dataset/"

# 2. Создаём подкаталог для метаданных /covid_dataset/metadata
make_dir(HDFS_METADATA_DIR)      # локально будет "covid_dataset/metadata/"

# 3. Создаём подкаталог для изображений /covid_dataset/images
make_dir(HDFS_IMAGES_DIR)        # локально "covid_dataset/images/"

# 4. Создаём подкаталог для обработанных данных /covid_dataset/processed
make_dir(HDFS_PROCESSED_DIR)     # локально "covid_dataset/processed/"


Создаём локальный каталог (эмуляция HDFS): covid_dataset
Создаём локальный каталог (эмуляция HDFS): covid_dataset/metadata
Создаём локальный каталог (эмуляция HDFS): covid_dataset/images
Создаём локальный каталог (эмуляция HDFS): covid_dataset/processed


In [19]:
# Определяем пути к подкаталогам с изображениями по укрупнённым диагнозам
HDFS_IMAGES_COVID_DIR = f"{HDFS_IMAGES_DIR}/covid"         # снимки пациентов с COVID-19
HDFS_IMAGES_PNEUMONIA_DIR = f"{HDFS_IMAGES_DIR}/pneumonia" # снимки пациентов с пневмонией
HDFS_IMAGES_NORMAL_DIR = f"{HDFS_IMAGES_DIR}/normal"       # нормальные снимки без патологии
HDFS_IMAGES_OTHER_DIR = f"{HDFS_IMAGES_DIR}/other"         # прочие случаи

# Создаём эти каталоги (в HDFS или локально — в зависимости от режима)
make_dir(HDFS_IMAGES_COVID_DIR)        # /covid_dataset/images/covid  → локально covid_dataset/images/covid
make_dir(HDFS_IMAGES_PNEUMONIA_DIR)    # /covid_dataset/images/pneumonia
make_dir(HDFS_IMAGES_NORMAL_DIR)       # /covid_dataset/images/normal
make_dir(HDFS_IMAGES_OTHER_DIR)        # /covid_dataset/images/other


Создаём локальный каталог (эмуляция HDFS): covid_dataset/images/covid
Создаём локальный каталог (эмуляция HDFS): covid_dataset/images/pneumonia
Создаём локальный каталог (эмуляция HDFS): covid_dataset/images/normal
Создаём локальный каталог (эмуляция HDFS): covid_dataset/images/other


In [20]:
# Кладём очищенный CSV-файл с метаданными в "HDFS"-каталог metadata
put_file(LOCAL_METADATA_CSV_PATH, HDFS_METADATA_DIR)        # metadata_cleaned.csv → covid_dataset/metadata/

# Кладём Parquet-файл с метаданными туда же
put_file(LOCAL_METADATA_PARQUET_PATH, HDFS_METADATA_DIR)    # metadata.parquet → covid_dataset/metadata/


Копируем файл metadata_cleaned.csv → covid_dataset/metadata/metadata_cleaned.csv (эмуляция HDFS)
Копируем файл metadata.parquet → covid_dataset/metadata/metadata.parquet (эмуляция HDFS)


In [21]:
# Словарь сопоставления укрупнённого диагноза и HDFS-папки с изображениями
DIAGNOSIS_TO_HDFS_DIR = {
    "COVID19": HDFS_IMAGES_COVID_DIR,        # COVID-случаи
    "PNEUMONIA": HDFS_IMAGES_PNEUMONIA_DIR,  # пневмонии
    "NORMAL": HDFS_IMAGES_NORMAL_DIR,        # нормальные снимки
    "OTHER": HDFS_IMAGES_OTHER_DIR           # все остальные
}

# Смотрим распределение укрупнённых диагнозов
print("Распределение укрупнённых диагнозов:")
print(metadata_clean["finding_unified"].value_counts())  # сколько записей в каждой категории

# Проверяем, есть ли пропуски в имени файла (filename)
missing_filenames = metadata_clean["filename"].isna().sum()  # считаем количество NaN в колонке filename
print("Пропусков в колонке 'filename':", missing_filenames)  # выводим результат


Распределение укрупнённых диагнозов:
finding_unified
COVID19      584
PNEUMONIA    242
OTHER        102
NORMAL        22
Name: count, dtype: int64
Пропусков в колонке 'filename': 0


In [24]:
# Счётчик загруженных файлов по категориям (для статистики)
uploaded_counts = {
    "covid": 0,
    "pneumonia": 0,
    "normal": 0,
    "other": 0
}

# Если папки с изображениями вообще нет — честно говорим об этом и не пытаемся ничего копировать
if not os.path.isdir(LOCAL_IMAGES_DIR):
    print(f"⚠️ Папка с изображениями '{LOCAL_IMAGES_DIR}' не найдена.")
    print("   Предполагается, что здесь лежит папка с рентген-снимками датасета.")
    print("   Загрузку изображений в HDFS пропускаем, но структура каталогов уже создана.")
else:
    # Если папка есть, пробуем реально разложить файлы по категориям
    for index, row in metadata_clean.iterrows():
        filename = row["filename"]                  # имя файла изображения
        diagnosis = row["finding_unified"]          # укрупнённый диагноз

        if pd.isna(filename):
            print(f"Строка {index}: пропущено, нет имени файла")
            continue

        # Выбираем целевую папку по диагнозу
        hdfs_target_dir = DIAGNOSIS_TO_HDFS_DIR.get(diagnosis, HDFS_IMAGES_OTHER_DIR)

        # Склеиваем локальный путь
        local_image_path = os.path.join(LOCAL_IMAGES_DIR, filename)

        # Проверяем, что файл существует
        if not os.path.exists(local_image_path):
            print(f"Строка {index}: файл {local_image_path} не найден, пропускаем")
            continue

        # Кладём файл (в HDFS или локально — в зависимости от режима)
        put_file(local_image_path, hdfs_target_dir)

        # Обновляем счётчик
        if hdfs_target_dir == HDFS_IMAGES_COVID_DIR:
            uploaded_counts["covid"] += 1
        elif hdfs_target_dir == HDFS_IMAGES_PNEUMONIA_DIR:
            uploaded_counts["pneumonia"] += 1
        elif hdfs_target_dir == HDFS_IMAGES_NORMAL_DIR:
            uploaded_counts["normal"] += 1
        else:
            uploaded_counts["other"] += 1

    print("Загрузка изображений завершена.")
    print("Статистика по загруженным файлам:")
    for category, count in uploaded_counts.items():
        print(f"{category}: {count}")


⚠️ Папка с изображениями 'images' не найдена.
   Предполагается, что здесь лежит папка с рентген-снимками датасета.
   Загрузку изображений в HDFS пропускаем, но структура каталогов уже создана.
